In [1]:
# DIRECTORY SET
import os
import sys
import traceback
from pathlib import Path
base_dir=Path(os.getcwd()).parent.parent
os.chdir(base_dir)
print(os.getcwd())

# ENVIRONMENT VARIABLES
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

# Enable autoreload
%load_ext autoreload
%autoreload 2

# Import custom modules
from destinyapp.models import StreamRecapData

from destinyapp.customlibrary import services
from destinyapp.customlibrary import utils
from destinyapp.customlibrary import controller

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


ValueError: mutable default <class 'destinyapp.customlibrary.services.RV.TimeNormalization'> for field time_normalization is not allowed: use default_factory

# Load Data

In [ ]:
# Input parameter
video_id="OqVH_MTBQ6k"

# load data from testing_visualization_annotation_results.json
import json
with open(f"testing_visualization_data_gen.json", "r") as f:
    data_gen=json.load(f)

annotated_results=data_gen["annotated_results"]
major_topics=data_gen["major_topics"]
minor_topics=data_gen["minor_topics"]

stream_recap_data=await utils.get_recap_data(video_id)

plot_segments, category_locations, color_dict = await services.RV.create_segments(stream_recap_data.linked_transcript, annotated_results, major_topics, stream_recap_data.transcript)

# Shove data into class

In [ ]:
plot_object=services.RV.PlotObject()

In [ ]:
# Define Segments
plot_object.segments=[]
for segment in plot_segments:
    segment["x"]=None
    seg_obj=services.RV.Segment(**segment)
    plot_object.segments.append(seg_obj)

In [ ]:
# Define Abstractions
plot_object.time_normalization.net_duration=plot_object.segments[-1].end-plot_object.segments[0].start
plot_object.time_normalization.start_offset=plot_object.segments[0].start
for segment in plot_object.segments:
    abstraction=plot_object.abstractions.get(segment.category, None)
    if abstraction:
        plot_object.abstractions[segment.category].width+=abstraction.width
    else:
        plot_object.abstractions[segment.category]=services.RV.Abstraction(width=segment.width, color=segment.color)

    segment.x=(((segment.start+segment.end)/2)/plot_object.time_normalization.net_duration)-plot_object.time_normalization.start_offset

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# Create the plot with a specific gray background
background_color=96
fig, ax = plt.subplots(figsize=(12, 10))
hex_background_color = '#%02x%02x%02x' % (int(background_color), int(background_color), int(background_color))
fig.patch.set_facecolor(hex_background_color)  # Set figure background to [96, 96, 96]
ax.set_facecolor(hex_background_color)  # Set axes background to [96, 96, 96]
target_plot_width=10

plt.close(fig)

In [ ]:
plot_object.segments[0].start

In [ ]:
plot_object.segments[-1].end

In [ ]:
# import defaultdict
from collections import defaultdict

# Plot BAR 
current_x = 0
category_info = defaultdict(lambda: {"total_width": 0, "segments": []})

bar_height = 1.5
bar_height = 1.0

clickable_area_x_offset=0.085
clickable_area_y_offset=0.1
clickable_y_height_multiplier=1.22
clickable_areas=[]
href_base="https://youtu.be/"+video_id+"?t="



for i, segment in enumerate(plot_object.segments):
    ax.add_patch(plt.Rectangle((current_x, 0), segment.width, bar_height, 
                            facecolor=segment.color, edgecolor='white'))
    current_x += segment.width

    # coord_list=[(current_x)*10, (10-(bar_height*clickable_y_height_multiplier)-clickable_area_y_offset)*10, (current_x+segment.width)*10, 10*(10-clickable_area_y_offset)]
    
    # href=href_base+str(int(segment.start))
    # clickable_area={"alt":href, "title":href, "href":href, "coords": coord_list}
    # clickable_areas.append(clickable_area)

total_width = current_x
print("Total width:", total_width)

In [ ]:
# Add white line extending upward from central circle
top_y = 100  # Adjust this value to change the length of the line
ax.plot([total_width/2, total_width/2], [0, top_y], 
        color='white', linewidth=3, solid_capstyle='round')  # Added white line

# Customize the plot
ax.set_xlim(0, total_width)
ax.set_ylim(0, 8)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')

# Display the plot
from IPython.display import display

display(fig)